# Square Main Cluster

In this notebook we will show how to get a square main cluster of the quasiperiodic lattice around an arbitrary point.

Previously we were able to generate approximately circular main clusters around an arbitrary point. In fact, in the notebook of the folder "Main_Cluster_Size" we showed how to get the average radius of thoses clusters.

However, for some algorithms it is convenient (if not necessary) to have a square and not circular main cluster.

Let's load the files that will be used in this notebook.

In [1]:
include("Operaciones_Basicas.jl")
include("Red_Dual.jl")
include("Separacion_Arreglos_X_Y.jl")
include("Generador_Vecindades_Pto_Arbitrario.jl")
include("voronoi.jl")
include("Algoritmo_Voronoi.jl")
include("Algoritmo_Cluster_Principal_Iterado.jl")
include("Algoritmo_Parche_Cuadrado.jl")

parche_Cuadrado (generic function with 2 methods)

We will use the package "Plots" to visualize the polygons of the Neighborhood and the Voronoi's polygons. So, let's load it.

In [2]:
using Plots
gr();

### Information about the Quasiperiodic Lattice

Create the information of the star vectors required by the GDM algorithm, which determine the symmetry of the quasiperiodic lattice. Also, generate the array that contains the alpha constants and the array with the average distance between stripes.

In [3]:
NSides = 5; #Symmetry of the quasiperiodic lattice

Star_Vectors = [[BigFloat(1),0]]; #Array wich will contain the Star Vectors

for i in 1:(NSides-1)
    push!(Star_Vectors, [cos((2*i)*pi/NSides), sin((2*i)*pi/NSides)]); #Complete the Star_Vector Array
end

Alphas_Array = fill(0.2, NSides); #Array of the alphas constant

Average_Distance_Stripes = fill(NSides/2, NSides); #Array with the average distance between stripes

To operate with this algorithm, we need the information about the almost-circular main clusters generated by the areas discrimination algorithm based in the GDM and the Voronoi's algorithm (check the notebook of the folder "Main_Cluster's_Polygons_(Areas_Algorithm)" to more information about this algorithm).

In the notebook contained in the folder "Main_Cluster_Size" we showed how to obtain that information and how to save it for future use. In this notebook, we must load that info and use it to generate our square main cluster.

To read the information we use the package "DelimitedFiles", so let's load it.

In [4]:
using DelimitedFiles

In [5]:
#Let's read the information of the almost-circular neighborhoods generated by the 
Patch_Information = readdlm("Informacion_Parche_N_5.csv", BigFloat);

The information contained in the "Patch_Information" array is:

1.- The margin of error in the integer numbers generated as the projection of the arbitrary point into the star vector.

2.- The size of a half side of the square in which the algorithm generate a random point inside it.

3.- Number of iterations of the areas algorithm used (see "Main_Cluster's_Polygons_(Areas_Algorithm)"

4.- The value for the area of the polygons that will be a discriminator value in the areas algorithm.

5.- Number of the clusters that were analyzed to get the Average Radius.

6.- Average radius of the almost-circular neighborhoods.

In [6]:
Patch_Information

6×1 Array{BigFloat,2}:
 2.0e+01                                                                             
 1.0e+07                                                                             
 2.0e+01                                                                             
 1.1999999999999999555910790149937383830547332763671875                              
 1.0e+02                                                                             
 3.918400076378164465840114871753030456602573394775390625000000000000000000000013e+01

The average radius, as his name implies, is an average parameter so the almost-circular main clusters that we will genrate doesn't fit perfectly inside that radius.

Because that we prefer going safe and consider as a "Safe Radius" the 80% of the value of the "Average Radius", this to ensure that inside the Safe Radius we will have polygons of the Main Clusters that fill that space.

i.e.

$$R_{S} = (0.8)(R_{A}),$$

with $R_{S}$ the Safe Radius and $R_{A}$ the Average Radius.

In [7]:
Average_Radius = Patch_Information[end]; #The value of the previously calculated "Average Radius"
Safe_Radius = 0.8*Average_Radius; #The Safe Radius value
println("The Safe Radius of the Main Cluster generated with the configuration of the Patch_Information array is: ")
println(Safe_Radius)

The Safe Radius of the Main Cluster generated with the configuration of the Patch_Information array is: 
3.134720061102531746684011276922830628427035637574536179997748187604084124657798e+01


### Square Patch Algorithm

Now, let's describe the algorithm to generate a square main cluster around an arbitrary point:

1.- Let's get an arbitrary point in the 2D space (That point will be the center of our Square Main Cluster).

2.- Let's create four almost-circular main cluster in the points:
$$P_{1} = C + \frac{\sqrt{2}}{2} \left( 1, 1 \right) R_{S},$$
$$P_{2} = C + \frac{\sqrt{2}}{2} \left( -1, 1 \right) R_{S},$$
$$P_{3} = C + \frac{\sqrt{2}}{2} \left( -1, -1 \right) R_{S},$$
$$P_{4} = C + \frac{\sqrt{2}}{2} \left( 1, -1 \right) R_{S},$$

3.- Let's eliminate all the repeated polygons in the regions were our almost-circular main clusters intersect.

4.- The squared main cluster will have a safe side length of 
$$L = \sqrt{8} R_{S},$$
with vertices in:
$$V_{1} = C + \sqrt{2} \left( 1, 1 \right) R_{S},$$
$$V_{2} = C + \sqrt{2} \left( -1, 1 \right) R_{S},$$
$$V_{3} = C + \sqrt{2} \left( -1, -1 \right) R_{S},$$
$$V_{4} = C + \sqrt{2} \left( 1, -1 \right) R_{S},$$

Let's generate an square main cluster with the previous algorithm information

In [8]:
SL = Patch_Information[2]; #Size of a half side of the square in which the algorithm generate a random point inside it
Reduction_Factor = 0.8; #The factor with which we multiple the Average Radius to generate the Safe Radius
X, Y, APoint = parche_Cuadrado(Patch_Information, SL, Reduction_Factor, Average_Distance_Stripes, Star_Vectors, Alphas_Array);

There is a total of 24123 polygons with possible duplicates.
There is a total of 16726 uniques polygons.


Let's visualize the obtained with the previous function

In [9]:
#Consume muchísimos recursos en graficar y visualizar el cluster "cuadrado" principal. Usar con SUMO CUIDADO.


plot()
for i in 1:4:length(X)
    plot!([X[i],X[i+1],X[i+2],X[i+3],X[i]],[Y[i],Y[i+1],Y[i+2],Y[i+3],Y[i]], markersize = 0.2, key = false, aspect_ratio=:equal, grid = false, color =:black)
end
plot!()
scatter!([APoint[1]], [APoint[2]], color = "red")
scatter!([APoint[1]+sqrt(2)*Safe_Radius], [APoint[2]+sqrt(2)*Safe_Radius], color = "red")
scatter!([APoint[1]-sqrt(2)*Safe_Radius], [APoint[2]+sqrt(2)*Safe_Radius], color = "red")
scatter!([APoint[1]-sqrt(2)*Safe_Radius], [APoint[2]-sqrt(2)*Safe_Radius], color = "red")
scatter!([APoint[1]+sqrt(2)*Safe_Radius], [APoint[2]-sqrt(2)*Safe_Radius], color = "red")
